# Laboratorium 5 - rekomendacje grupowe

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas scipy matplotlib`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds


from random import choice, sample
from statistics import mean, stdev

In [2]:
PATH = 'ml-latest-small'

In [3]:
# wczytujemy oceny uzytkownikow i obliczamy (za pomoc dekompozycji macierzy) wszystkie przewidywane oceny filmow

def read_ratings(path, k=600, scale_factor=2.0, print_stats=True):
    # idea: https://www.kaggle.com/code/indralin/movielens-project-1-2-collaborative-filtering
    reviews = pd.read_csv(f'{path}/ratings.csv', names=['userId', 'movieId', 'rating', 'time'], delimiter=',', engine='python', skiprows=1)
    
    reviews.drop(['time'], axis=1, inplace=True)
    reviews_no, _ = reviews.shape
    reviews_matrix = reviews.pivot(index='userId', columns='movieId', values='rating')
    movies = reviews_matrix.columns
    users = reviews_matrix.index
    users_no, movies_no = reviews_matrix.shape
    print(f'Got {reviews_no} reviews for {movies_no} movies and {users_no} users.')

    user_ratings_mean = np.nanmean(reviews_matrix.values, axis=1)
    normalized_reviews_matrix = np.nan_to_num(reviews_matrix.values - user_ratings_mean.reshape(-1, 1), 0.0)

    U, sigma, Vt = svds(normalized_reviews_matrix, k=k)
    sigma = np.diag(sigma)
    predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1).clip(0.5, 5.0)
    mean_square_error = np.nanmean(np.square(predicted_ratings - reviews_matrix.values))
    std_square_error = np.nanstd(np.square(predicted_ratings - reviews_matrix.values))
    print(f'Reviews prediction mean square error = {mean_square_error}')
    print(f'Reviews prediction standatd deviation of square error = {std_square_error}')

    if print_stats:
        stats = [
            ('metric', 'dataset', 'prediction'),
            ('avg', np.nanmean(reviews_matrix), np.mean(predicted_ratings)),
            ('st_dev', np.nanstd(reviews_matrix), np.std(predicted_ratings)),
            ('median', np.nanmedian(reviews_matrix), np.median(predicted_ratings)),
            ('p25', np.nanquantile(reviews_matrix, 0.25), np.quantile(predicted_ratings, 0.25)),
            ('p75', np.nanquantile(reviews_matrix, 0.75), np.quantile(predicted_ratings, 0.75))
        ]
        print('Stats (for raings in original range [0.5, 5.0]):')
        print('\n'.join([str(s) for s in stats]))

    rounded_predictions = np.rint(scale_factor * predicted_ratings) # cast values to {1, 2, ..., 10}
    return pd.DataFrame(data=rounded_predictions, index=list(users), columns=list(movies))
    
ratings = read_ratings(PATH)
# dostep do danych:
# ratings[movieId][userId] pobiera 1 wartosc
# ratings.loc[:, movieId] pobiera wektor dla danego filmu
# ratings.loc[userId, :] pobiera wektor dla danego uzytkownika
ratings

Got 100836 reviews for 9724 movies and 610 users.
Reviews prediction mean square error = 1.6577787842924763e-05
Reviews prediction standatd deviation of square error = 0.0007928950536518541
Stats (for raings in original range [0.5, 5.0]):
('metric', 'dataset', 'prediction')
('avg', np.float64(3.501556983616962), np.float64(3.6572223377474))
('st_dev', np.float64(1.0425240696180562), np.float64(0.4954623756097101))
('median', np.float64(3.5), np.float64(3.7052240088117694))
('p25', np.float64(3.0), np.float64(3.357451718316403))
('p75', np.float64(4.0), np.float64(3.9999816268830015))


,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1,8.0,9.0,8.0,9.0,9.0,8.0,9.0,9.0,9.0,9.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
2,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
3,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
4,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
5,8.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,5.0,7.0,7.0,7.0,7.0,7.0,5.0,7.0,7.0,7.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
607,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,...,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
608,5.0,4.0,4.0,6.0,6.0,6.0,6.0,6.0,6.0,8.0,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
609,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,8.0,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0


In [4]:
# wczytujemy nazwy filmow i kategorie

movies_metadata = pd.read_csv('ml-latest-small/movies.csv').set_index('movieId')
movies_metadata

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


In [5]:
# wczytujemy przykladowe grupy uzytkownikow
groups = pd.read_csv('groups.csv').values.tolist()
groups

[[111, 307, 474, 599, 414],
 [469, 182, 232, 448, 600],
 [508, 581, 497, 402, 566],
 [300, 515, 245, 568, 507],
 [2, 371, 252, 518, 37],
 [269, 360, 469, 287, 308],
 [243, 527, 418, 118, 370],
 [186, 559, 327, 553, 314]]

In [6]:
# przygotowujemy funkcje pomocnicza

def describe_group(group, N=10):
    print(f'\n\nUser ids: {group}')
    group_size = len(group)
    
    mean_stdev = ratings.loc[group].std(axis=0).mean()
    median_stdev = ratings.loc[group].std(axis=0).median()
    std_stdev = ratings.loc[group].std(axis=0).std()
    print(f'\nMean ratings deviation: {mean_stdev}')
    print(f'Median ratings deviation: {median_stdev}')
    print(f'Standard deviation of ratings deviation: {std_stdev}')
    
    average_scores = ratings.iloc[group].mean(axis=0)
    average_scores = average_scores.sort_values()
    best_movies = [(movies_metadata['title'][movie_id], average_scores[movie_id]) for movie_id in list(average_scores[-N:].index)]
    worst_movies = [(movies_metadata['title'][movie_id], average_scores[movie_id]) for movie_id in list(average_scores[:N].index)]
    
    print('\nBest movies:')
    for movie, score in best_movies[::-1]:
        print(f'{movie}, {score}*')
    print('\nWorst movies:')
    for movie, score in worst_movies:
        print(f'{movie}, {score}*')

describe_group(groups[5])



User ids: [269, 360, 469, 287, 308]

Mean ratings deviation: 1.125914957457979
Median ratings deviation: 1.0954451150103321
Standard deviation of ratings deviation: 0.17836724055768716

Best movies:
Forrest Gump (1994), 8.2*
Toy Story (1995), 8.2*
Braveheart (1995), 8.0*
Willy Wonka & the Chocolate Factory (1971), 8.0*
Terminator 2: Judgment Day (1991), 7.8*
Schindler's List (1993), 7.8*
Shawshank Redemption, The (1994), 7.6*
Twelve Monkeys (a.k.a. 12 Monkeys) (1995), 7.6*
Nixon (1995), 7.6*
Dances with Wolves (1990), 7.6*

Worst movies:
Broken Arrow (1996), 5.2*
Cable Guy, The (1996), 5.4*
Sleepy Hollow (1999), 5.4*
The Devil's Advocate (1997), 5.4*
Nutty Professor, The (1996), 5.6*
Mission: Impossible (1996), 5.6*
Matrix Revolutions, The (2003), 5.8*
Cheech & Chong's The Corsican Brothers (1984), 5.8*
Hellbound: Hellraiser II (1988), 5.8*
Masquerade (1988), 5.8*


## Część 2. - algorytmy proste

In [7]:
# zdefiniujmy interfejs dla wszystkich algorytmow rekomendacyjnych

class Recommender:
    def recommend(self, movies, ratings, group, size):
        pass


# jako pierwszy zaimplementujemy algorytm losowy - dla porownania
    
class RandomRecommender(Recommender):
    def __init__(self):
        self.name = 'random'
        
    def recommend(self, movies, ratings, group, size):
        return sample(movies, size)

In [8]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen

class AverageRecommender(Recommender):
    def __init__(self):
        self.name = 'average'
    
    def recommend(self, movies, ratings, group, size):
        selected_rows = ratings.loc[ratings.index.isin(group)]
        movies_averages = dict()

        for column in selected_rows.columns:
            movies_averages[column] = selected_rows[column].mean()

        best_movies = [score[0] for score in sorted(movies_averages.items(), key=lambda x: x[1], reverse=True)[:size]]
        
        return best_movies       

In [9]:
# algorytm rekomendujacy filmy o najwyzszej sredniej ocen,
#   ale rownoczesnie wykluczajacy te filmy, ktore otrzymaly choc jedna ocene ponizej thresholdu

class AverageWithoutMiseryRecommender(Recommender):
    def __init__(self, score_threshold):
        self.name = 'average_without_misery'
        self.score_threshold = score_threshold
        
    def recommend(self, movies, ratings, group, size):
        selected_rows = ratings.loc[ratings.index.isin(group)]
        movies_averages = dict()

        for column in selected_rows.columns:
            if (selected_rows[column] < self.score_threshold).any():
                continue
            else:
                movies_averages[column] = selected_rows[column].mean()

        best_movies = [score[0] for score in sorted(movies_averages.items(), key=lambda x: x[1], reverse=True)[:size]]
        
        return best_movies

In [10]:
# algorytm uwzgledniajacy preferencje tylko jednego uzytkownika w kazdej iteracji

class FairnessRecommender(Recommender):
    def __init__(self):
        self.name = 'fairness'
        
    def recommend(self, movies, ratings, group, size):
        selected_rows = ratings.loc[ratings.index.isin(group)]
        best_movies = []
        group_size = len(group)

        for i in range(size):
            user = group[i % group_size]
            found = False
            counter = 1
            
            while not found and counter < size:
                max_columns = selected_rows.loc[user].nlargest(counter).index

                for column in max_columns:
                    if column not in best_movies:
                        best_movies.append(column)
                        found = True
                        break

                counter += 1

        return best_movies

In [11]:
# wybrany algorytm wyborczy (dyktatura, Borda, Copeland)

class VotingRecommender(Recommender):
    def __init__(self):
        self.name = 'borda'
    
    def recommend(self, movies, ratings, group, size):
        selected_rows = ratings.loc[ratings.index.isin(group)]
        movie_scores = dict()

        for column in selected_rows.columns:
            movie_scores[column] = 0

        for user in group:
            user_ratings = selected_rows.loc[user]
            ranked_movies = user_ratings.sort_values(ascending=False).index.tolist()

            for rank, movie_id in enumerate(ranked_movies):
                movie_scores[movie_id] += (len(ranked_movies) - rank)

        best_movies = [score[0] for score in sorted(movie_scores.items(), key=lambda x: x[1], reverse=True)[:size]]
        
        return best_movies

In [12]:
# algorytm zachlanny, aproksymujacy metode Proportional Approval Voting
#   w kazdej iteracji wybieramy ten film, ktory najbardziej zwieksza zadowolenie zgodnie z punktacja PAV

class ProportionalApprovalVotingRecommender(Recommender):
    def __init__(self, threshold):
        self.threshold = threshold
        self.name = 'PAV'
        
    def recommend(self, movies, ratings, group, size):
        selected_rows = ratings.loc[ratings.index.isin(group)]
        scores = {user: 1 for user in group}
        best_movies = []

        for _ in range(size):
            movies_scores = {movie: 1 for movie in movies}

            for movie in [col for col in selected_rows.columns if col not in best_movies]:
                for user in group:
                    if selected_rows.loc[user, movie] > self.threshold:
                        movies_scores[movie] += (1 / scores[user])

            best_movie = sorted(movies_scores.items(), key=lambda x: x[1], reverse=True)[0][0]
            best_movies.append(best_movie)

            for user in group:
                if selected_rows.loc[user, best_movie] > self.threshold:
                    scores[user] += 1
                    
        return best_movies

## Część 3. - funkcje celu

In [13]:
# dwie funkcje pomocnicze:
#  - znajdujaca ulubione filmy danego uzytkownika
#  - obliczajaca sume ocen wystawionych przez uzytkownika wszystkim filmom w rekomendacji

def top_n_movies_for_user(ratings, movies, user_id, n):
    return ratings.loc[user_id].nlargest(n).index

def total_score(recommendation, user_id, ratings):
    return ratings.loc[user_id, recommendation].sum()

In [14]:
# funkcja obliczajaca zadowolenie pojedynczego uzytkownika
#  - iloraz zadowolenia z wygenerowanej rekomendacji oraz zadowolenia z hipotetycznej rekomendacji idealnej
def overall_user_satisfaction(recommendation, user_id, movies, ratings):
    recommendation_score = total_score(recommendation, user_id, ratings)
    best_score = total_score(top_n_movies_for_user(ratings, movies, user_id, len(recommendation)), user_id, ratings)

    if best_score == 0:
        return 1
    else:
        return recommendation_score / best_score

# funkcja celu - srednia z zadowolenia wszystkich uzytkownikow w grupie
def overall_group_satisfaction(recommendation, group, movies, ratings):
    if len(group) == 0:
        return 0
    
    score = 0

    for user in group:
        score += overall_user_satisfaction(recommendation, user, movies, ratings)

    return score / len(group)

# funkcja celu - roznica miedzy maksymalnym i minimalnym zadowolenie w grupie
def group_disagreement(recommendation, group, movies, ratings):
    if len(group) == 0:
        return 0
    
    max = 0
    min = float('inf')

    for user in group:
        score = overall_user_satisfaction(recommendation, user, movies, ratings)
        
        if score > max:
            max = score
            
        if score < min:
            min = score

    return max - min

## Część 4. - Sequential Hybrid Aggregation

In [15]:
# algorytm balansujacy pomiedzy wyborem elementow o najwyzszej sredniej ocen
#   i o najwyzszej minimalnej ocenie
#   wyliczajacy w kazdej iteracji parametr alfa - jak na wykladzie
class SequentialHybridAggregationRecommender(Recommender):
    def __init__(self):
        self.name = 'sequential_hybrid_aggregation'
    
    def recommend(self, movies, ratings, group, size):
        selected_rows = ratings.loc[ratings.index.isin(group)]
        best_movies = []

        for _ in range(size):
            movies_scores = {movie: 0 for movie in movies}
            alpha = 0

            for movie in [col for col in selected_rows.columns if col not in best_movies]:
                avg_score = selected_rows[movie].mean()
                min_score = selected_rows[movie].min()

                movies_scores[movie] = (1 - alpha) * avg_score + alpha * min_score

            best_movie = sorted(movies_scores.items(), key=lambda x: x[1], reverse=True)[0][0]
            best_movies.append(best_movie)
            alpha = group_disagreement(best_movies, group, movies, ratings)
                    
        return best_movies

## Część 5. - porównanie algorytmów

In [16]:
recommenders = [
    RandomRecommender(),
    AverageRecommender(),
    AverageWithoutMiseryRecommender(5),
    FairnessRecommender(),
    VotingRecommender(),
    ProportionalApprovalVotingRecommender(5),
    SequentialHybridAggregationRecommender()
]

recommendation_size = 10
movies = movies_metadata.index.tolist()

# dla kazdego algorytmu:
#  - wygenerujmy jedna rekomendacje dla kazdej grupy
#  - obliczmy wartosci obu funkcji celu dla kazdej rekomendacji
#  - wypiszmy wyniki na konsole

for recommender in recommenders:
    print(f'\n\n\n{recommender.name}')
    group_satisfaction = []
    group_disagreements = []

    for group in groups:
        recommendation = recommender.recommend(movies, ratings, group, recommendation_size)
        group_satisfaction.append(overall_group_satisfaction(recommendation, group, movies, ratings))
        group_disagreements.append(group_disagreement(recommendation, group, movies, ratings))
        
    print(f'\nGroup satisfaction: {[float(round(s, 2)) for s in group_satisfaction]}')
    print(f'Average group satisfaction: {round(mean(group_satisfaction), 4)}')

    print(f'\nGroup disagreement: {[float(round(d, 2)) for d in group_disagreements]}')
    print(f'Average group disagreement: {round(mean(group_disagreements), 4)}')




random

Group satisfaction: [0.6, 0.66, 0.76, 0.87, 0.83, 0.63, 0.78, 0.76]
Average group satisfaction: 0.7375

Group disagreement: [0.21, 0.1, 0.26, 0.24, 0.12, 0.29, 0.27, 0.28]
Average group disagreement: 0.221



average

Group satisfaction: [0.89, 0.89, 0.86, 0.93, 0.88, 0.78, 0.89, 0.87]
Average group satisfaction: 0.8751

Group disagreement: [0.23, 0.25, 0.21, 0.11, 0.13, 0.29, 0.15, 0.17]
Average group disagreement: 0.1908



average_without_misery

Group satisfaction: [0.89, 0.89, 0.82, 0.93, 0.88, 0.78, 0.89, 0.87]
Average group satisfaction: 0.8702

Group disagreement: [0.23, 0.25, 0.3, 0.11, 0.13, 0.29, 0.15, 0.17]
Average group disagreement: 0.2025



fairness

Group satisfaction: [0.76, 0.81, 0.84, 0.91, 0.87, 0.7, 0.85, 0.82]
Average group satisfaction: 0.82

Group disagreement: [0.14, 0.16, 0.2, 0.14, 0.08, 0.32, 0.15, 0.19]
Average group disagreement: 0.1722



borda

Group satisfaction: [0.88, 0.87, 0.76, 0.87, 0.83, 0.66, 0.81, 0.76]
Average group satisfaction: 0.